# Import data from JSON files

Load structured data from JSON files into Pixeltable tables for processing and analysis.

## Problem

You have data in JSON format—from APIs, exports, or application logs. You need to load this data for processing with AI models or combining with other data sources.

| Source | Records | Use case |
|--------|---------|----------|
| api_response.json | 1,000 | Analyze API data |
| user_events.json | 50,000 | Process event logs |
| products.json | 500 | Enrich with AI descriptions |

## Solution

**What's in this recipe:**
- Import JSON files directly into tables
- Import from URLs (APIs, remote files)
- Handle nested JSON structures

You use `pxt.create_table()` with a `source` parameter to create a table from a JSON file or URL. The JSON must be an array of objects, where each object becomes a row.

### Setup

In [ ]:
%pip install -qU pixeltable

In [ ]:
import pixeltable as pxt
import json
import tempfile
from pathlib import Path

### Create sample JSON file

First, create a sample JSON file to demonstrate the import process:

In [ ]:
# Create sample JSON data (array of objects)
sample_data = [
    {'id': 1, 'title': 'Introduction to ML', 'author': 'Alice', 'tags': ['ml', 'intro'], 'rating': 4.5},
    {'id': 2, 'title': 'Deep Learning Basics', 'author': 'Bob', 'tags': ['dl', 'neural'], 'rating': 4.8},
    {'id': 3, 'title': 'NLP Fundamentals', 'author': 'Carol', 'tags': ['nlp', 'text'], 'rating': 4.2},
    {'id': 4, 'title': 'Computer Vision', 'author': 'Dave', 'tags': ['cv', 'images'], 'rating': 4.6},
    {'id': 5, 'title': 'Reinforcement Learning', 'author': 'Eve', 'tags': ['rl', 'agents'], 'rating': 4.3}
]

# Save to temporary JSON file
temp_dir = tempfile.mkdtemp()
json_path = Path(temp_dir) / 'articles.json'

with open(json_path, 'w') as f:
    json.dump(sample_data, f, indent=2)
)

### Import JSON file

Use `create_table` with `source` to create a table directly from a JSON file:

In [ ]:
# Create a fresh directory
pxt.drop_dir('json_demo', force=True)
pxt.create_dir('json_demo')

In [ ]:
# Import JSON file into a new table
# Note: source accepts Path objects or URLs for JSON files
articles = pxt.create_table(
    'json_demo.articles',
    source=json_path  # Path object works directly
)

In [ ]:
# View imported data
articles.collect()

### Import from URL

You can import JSON directly from a URL—useful for APIs and remote data:

In [ ]:
# Import from a public JSON URL
# Using JSONPlaceholder API as an example
posts = pxt.create_table(
    'json_demo.posts',
    source='https://jsonplaceholder.typicode.com/posts'
)

In [ ]:
# View first few rows
posts.head(5)

### Import from Python dictionaries

Use `create_table` with a list of dictionaries as `source`—useful when you have data in memory:

In [ ]:
# Import from a list of dictionaries
events = [
    {'event': 'page_view', 'user_id': 101, 'timestamp': '2024-01-15T10:30:00'},
    {'event': 'click', 'user_id': 101, 'timestamp': '2024-01-15T10:31:00'},
    {'event': 'purchase', 'user_id': 102, 'timestamp': '2024-01-15T10:32:00'},
]

event_table = pxt.create_table('json_demo.events', source=events)

In [ ]:
# View imported events
event_table.collect()

### Add computed columns

Once imported, you can enrich the data with computed columns:

In [ ]:
# Add a computed column combining title and author
articles.add_computed_column(
    summary=articles.title + ' by ' + articles.author
)

In [ ]:
# View with computed column
articles.select(articles.title, articles.author, articles.summary).collect()

## Explanation

**JSON format requirements:**

The JSON file must contain an array of objects at the top level:

```json
[
  {"col1": "value1", "col2": 123},
  {"col1": "value2", "col2": 456}
]
```

**Source types supported:**

| Source | Example |
|--------|---------|
| JSON file path | `source='/path/to/data.json'` |
| JSON URL | `source='https://api.example.com/data'` |
| List of dicts | `source=[{'a': 1}, {'a': 2}]` |

**Nested JSON handling:**

Nested objects and arrays are stored as JSON columns. You can access nested fields using Pixeltable's JSON path syntax in computed columns.

## See also

- [Import CSV files](https://docs.pixeltable.com/howto/cookbooks/data/data-import-csv) - For CSV and Excel imports
- [Import Parquet files](https://docs.pixeltable.com/howto/cookbooks/data/data-import-parquet) - For Parquet data
- [Extract fields from JSON](https://docs.pixeltable.com/howto/cookbooks/core/workflow-json-extraction) - Parse LLM response fields